# Day 17: Pyroclastic Flow 

It's basically tetris

Rocks: 5 shapes
```
rock 0
####

rock 1
.#.
###
.#.

rock 2
..#
..#
###

rock 3
#
#
#
#

rock 4
##
##
```

The tall, vertical chamber is exactly seven units wide. 

Each rock appears so that its left edge is two units away from the left wall and its bottom edge is three units above the highest rock in the room (or the floor, if there isn't one).

After a rock appears, it alternates between being pushed by a jet of hot gas one unit (in the direction indicated by the next symbol in the jet pattern) and then falling one unit down.


Result we're looking for is *height of rock tower after 2022 rocks have falled*


In [27]:
from pprint import pprint
#test data
testJetPattern = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

class Rock:
    def __init__(self, ascii:str):
        self.ascii = ascii
        self.height = 0
        self.rows = []
        for row in ascii.splitlines():
            self.height += 1
            self.width = len(row)
            line = []
            for c in list(row):
                if c == '#':
                    line.append(1)
                else:
                    line.append(0)
            self.rows.insert(0,line) #input in pictorial view.. so we need to flip the rows... i.e. last row becomes bottom row of our shape

    @classmethod
    def newShape(cls,shape:int):
        match shape:
            case 0:
                rock = '####'
            case 1:
                rock = """.#.
###
.#."""
                
            case 2:
                rock = """..#
..#
###"""

            case 3:
                rock = """#
#
#
#"""

            case 4:
                rock = """##
##"""
            case _:
                raise Exception('invalid shape number')
        return cls(rock)

class Game:
    def __init__(self):
        self.highestRock = 0 #start with the floor
        self.gridWidth = 7
        self.shapeStartLeftPad = 2
        self.shapeStartVerticalPad = 3
        self.currentShape = -1 
        self.shapeCount = 0
        self.rows:[[int]] = [[0] * self.gridWidth] #2D grid, row first (starting at floor) then column... 0=empty, 1=rock
        self.shapeRows = [] #2D grid to hold the sprite of the current shape... we can just chop off rows at the head to have it fall

    def nextShape(self):
        #make the next shape fall
        self.currentShape = (self.currentShape + 1) % 5
        self.shapeCount += 1
        rock = Rock.newShape(self.currentShape)

        #need to add three rows ABOVE THE CURRENT HIGHEST ROCK + height of shape... need the shapes as objects
        rowsHeight = len(self.rows)
        rowsToAdd = self.highestRock + 3 - rowsHeight + rock.height
        for _ in range(rowsToAdd):
            self.rows.append([0] * self.gridWidth)
        rowsHeight = len(self.rows)

        #need to build shapeRows and place the shape in the right place
        self.shapeRows = []
        for _ in range(rowsHeight):
            self.shapeRows.append([0] * self.gridWidth)
        for r in range(rock.height):
            for c in range(rock.width):
                rockValue = rock.rows[r][c]
                if rockValue == 1:
                    rr = r + self.highestRock + 3
                    cc = c + self.shapeStartLeftPad
                    self.shapeRows[rr][cc] = 1

    def shiftDown(self)->bool:
        #shift the shape down1 and return true... or, if this would cause a collision then return false
        emptyRow = [0] * self.gridWidth
        #check for being at bottom
        if self.shapeRows[0] != emptyRow:
            return False
        self.shapeRows.pop(0)
        noCollision = True
        for r in range(len(self.shapeRows)):
            if self.shapeRows[r]!=emptyRow and noCollision: #skip over the empty rows
                for c in range(self.gridWidth):
                    shapeValue = self.shapeRows[r][c]
                    gridValue = self.rows[r][c]
                    if shapeValue==1 and gridValue==1:
                        noCollision = False
                        break
        return noCollision
    
    def collision(self,trialShapeRows)->bool:
        #return true if the trialShapeRow would result in collision
        emptyRow = [0] * self.gridWidth
        collision = False
        for r in range(len(self.shapeRows)):
            if self.shapeRows[r]!=emptyRow and not collision: #skip over the empty rows
                for c in range(self.gridWidth):
                    shapeValue = self.shapeRows[r][c]
                    gridValue = self.rows[r][c]
                    if shapeValue==1 and gridValue==1:
                        collision = True
                        break
        return collision

    #OMG... the left or right shift could cause a collision... so need to check for this condition as well... and if it would collide, we don't move.
    def shiftLeft(self)->bool:
        #need to know where the most left part is
        leftmost = self.gridWidth
        for r in range(len(self.shapeRows)):
            for c in range(self.gridWidth):
                shapeValue = self.shapeRows[r][c]
                if shapeValue == 1:
                    leftmost = min(leftmost, c)
                    break
        if leftmost != 0:
            #can shift left...but need to check for collisions
            trailShapeRows = self.shapeRows.copy()

            for r in range(len(self.shapeRows)):
                for c in range(self.gridWidth):
                    if c + 1 < self.gridWidth:
                        trailShapeRows[r][c] = self.shapeRows[r][c+1]
                    else:
                        trailShapeRows[r][c] = 0
            if not self.collision(trailShapeRows):
                self.shapeRows = trailShapeRows
                return True
            else:
                return False
        else:
            return False

    def shiftRight(self):
        #need to know where right most part is
        rightmost = 0
        for r in range(len(self.shapeRows)):
            for c in range(self.gridWidth):
                shapeValue = self.shapeRows[r][c]
                if shapeValue == 1:
                    rightmost = max(rightmost, c)
        if rightmost != self.gridWidth-1:
            #can shift right
            trailShapeRows = self.shapeRows.copy()
            for r in range(len(self.shapeRows)):
                for c in range(self.gridWidth-1,-1,-1):
                    if c==0:
                        trailShapeRows[r][c] = 0
                    else:
                        trailShapeRows[r][c] = self.shapeRows[r][c-1]
            if not self.collision(trailShapeRows):
                self.shapeRows = trailShapeRows
                return True
            else:
                return False
        else:
            return False

    def solidifyShape(self):
        emptyRow = [0] * self.gridWidth
        for r in range(len(self.shapeRows)):
            if self.shapeRows[r]!=emptyRow : #skip over the empty rows
                for c in range(self.gridWidth):
                    shapeValue = self.shapeRows[r][c]
                    gridValue = self.rows[r][c]
                    if shapeValue==1:
                        self.rows[r][c] = 1

    def run(self, jetpatternStr:str, numRocks:int)->int:
        #return the height of the tower after x rocks
        jetpattern = list(jetpatternStr)
        self.nextShape()
        #alternates between being pushed by a jet of hot gas one unit (in the direction indicated by the next symbol in the jet pattern) and then falling one unit down
        while self.shapeCount <= numRocks+1:
            shiftDir = jetpattern.pop(0)
            if shiftDir == '>':
                self.shiftRight()
            elif shiftDir == '<':
                self.shiftLeft()
            else:
                raise Exception('Unexpected jet')
            if not self.shiftDown():
                #shape can't be dropped and needs to be solidified
                self.solidifyShape()
                print('After ' + str(self.shapeCount) + ' rocks, tower height is '+str(self.highestRock))
                self.nextShape()
        return self.highestRock
                


##unit tests
# r0 = Rock.newShape(0)
# print(r0.rows)
# r2 = Rock.newShape(2)
# print(r2.rows)
# print(r2.rows[0][2])


# g = Game()
# for i in range(4):
#     print('----')
#     g.nextShape()
#     pprint(g.shapeRows)
#     print(g.shiftDown())
#     print(g.shiftRight())
#     print(g.shiftRight())
#     print(g.shiftRight())
#     print(g.shiftRight())
#     print(g.shiftRight())
#     pprint(g.shapeRows)


#tests
g = Game()
print(g.run(testJetPattern,2022))



After 1 rocks, tower height is 0
After 2 rocks, tower height is 0
After 3 rocks, tower height is 0
After 4 rocks, tower height is 0
After 5 rocks, tower height is 0
After 6 rocks, tower height is 0
After 7 rocks, tower height is 0
After 8 rocks, tower height is 0
After 9 rocks, tower height is 0
After 10 rocks, tower height is 0
After 11 rocks, tower height is 0
After 12 rocks, tower height is 0
After 13 rocks, tower height is 0
After 14 rocks, tower height is 0
After 15 rocks, tower height is 0
After 16 rocks, tower height is 0
After 17 rocks, tower height is 0
After 18 rocks, tower height is 0
After 19 rocks, tower height is 0
After 20 rocks, tower height is 0
After 21 rocks, tower height is 0
After 22 rocks, tower height is 0
After 23 rocks, tower height is 0
After 24 rocks, tower height is 0
After 25 rocks, tower height is 0
After 26 rocks, tower height is 0
After 27 rocks, tower height is 0
After 28 rocks, tower height is 0
After 29 rocks, tower height is 0
After 30 rocks, tower h

IndexError: pop from empty list